In [1]:
import numpy as np
import pandas as pd
import joblib
import xgboost as xgb

def project_2_scoring(data):
    # Load the saved XGBoost model and preprocessing objects
    model = joblib.load('/Users/nathandhawale/Documents/School/Spring24/BUAN 6341 (Applied Machine Learning)/Project 2/project_artifacts/models/xgb_model.pkl')
    target_encoder = joblib.load('/Users/nathandhawale/Documents/School/Spring24/BUAN 6341 (Applied Machine Learning)/Project 2/project_artifacts/encoders/target_encoder.pkl')
    features_info = joblib.load('/Users/nathandhawale/Documents/School/Spring24/BUAN 6341 (Applied Machine Learning)/Project 2/project_artifacts/features/features_list.pkl')

    # Preprocess data
    categorical_cols = ['City', 'State', 'Bank', 'BankState']
    data_encoded = target_encoder.transform(data[categorical_cols])
    data.drop(categorical_cols, axis=1, inplace=True)
    data = pd.concat([data, data_encoded], axis=1)

    # Engineer features as per the original training setup
    data['Employee_Job_Creation_Ratio'] = data['NoEmp'] / (data['CreateJob'] + 1)
    data['Log_DisbursementGross'] = np.log(data['DisbursementGross'] + 1)
    data['State_Bank_Combined'] = data['State'].astype(str) + "_" + data['Bank'].astype(str)
    data['Urban_Rural_Ratio'] = data['UrbanRural'] / (data['NoEmp'] + 1)
    data['Loan_Size_Category'] = pd.cut(data['DisbursementGross'], bins=3, labels=["small", "medium", "large"])
    data['Approval_Disbursement_Diff'] = data['GrAppv'] - data['SBA_Appv']
    data['Employee_Per_Retained_Job'] = data['NoEmp'] / (data['RetainedJob'] + 1)
    data['High_Employee_Flag'] = (data['NoEmp'] > data['NoEmp'].median()).astype(int)
    data['NewExist_UrbanRural_Interaction'] = data['NewExist'] * data['UrbanRural']
    data['Sqrt_DisbursementGross'] = np.sqrt(data['DisbursementGross'])

    # Select only the features used by the model
    data_for_scoring = data[features_info]

    # Predict using the XGBoost model
    dmatrix = xgb.DMatrix(data_for_scoring)
    predictions_proba = model.predict(dmatrix, output_margin=False, ntree_limit=model.best_ntree_limit)

    # Format and return the results
    results_df = pd.DataFrame({
        'index': data.index,
        'label': (predictions_proba > 0.5).astype(int),
        'probability_0': 1 - predictions_proba,
        'probability_1': predictions_proba
    })

    return results_df
